In [ ]:
# Insight gain pipeline:
# 1. Определяем тренды (по базе данных)
# 2. По тренду определяем инсайты
# 3. Инсайт получается следующим образом:
# 4. По тренду определяем топ-10 новостей c vc.ru, habr, других ресурсов
# Всё.


In [13]:
from bs4 import BeautifulSoup
import requests
import logging


# Parser constants
LOGGING_LEVELS = {
    "debug": logging.DEBUG,
    "info": logging.INFO,
    "warning": logging.WARNING,
    "error": logging.ERROR,
    "critical": logging.CRITICAL,
}

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    level=LOGGING_LEVELS["info"],
)


def get_html(url: str) -> str:
    """
    Get html text from url
    :param url: url
    :return: html text
    """
    try:
        r = requests.get(url)
        r.raise_for_status()
        return r.text
    except (requests.RequestException, ValueError):
        logging.error("Network error")
        return ""


In [14]:
# vc.ru
def get_vc_insights(trend: str) -> list[dict]:
    """
    Get insights from vc.ru
    :param trend: trend
    :return: list of insights
    """
    insights = []
    html = get_html(f"https://vc.ru/search/v2/content/relevant?query={trend}")
    if html:
        soup = BeautifulSoup(html, "html.parser")
        insights = []
        insights_html = soup.find("div", {'class':'search-feed'}).findChildren("div", {'class':'content-feed'}, recursive=False)
        logging.info(f"vc: {len(insights)} insights found.")
        for item in insights_html:
            insights.append(
                {
                    "source": "vc.ru",
                    "title": item.find("div", class_="content-title").text.replace("\n", "").strip(),
                    "url": item.find("a", class_="content-link").get("href"),
                    "summary": item.find("p").text.replace("\xa0", " ").replace("<mark>", " ").replace("</mark>", " ").strip(),
                }
            )
    return insights


get_vc_insights("nft")[:2]

[{'source': 'vc.ru',
  'title': 'Что такое NFT',
  'url': 'https://vc.ru/crypto/214497-chto-takoe-nft',
  'summary': 'Cryptopunks - первый NFT коллектибл LarvaLabs Интерфейс MetaMaskNFT называют главным трендом 2021 года в блокчейне. Новостей из мира NFT все больше. Сделаю обзор топовых NFT проектов. Накину план, какие сделать первые шаги в NFT. NFT Окей, раз это аббревиатура, давайте начнем с расшифровки.  NFT = Non Fungible Token. Так и появились NFT. Теперь, благодаря NFT — нет.Что такое NFT'},
 {'source': 'vc.ru',
  'title': 'NFT для новичков',
  'url': 'https://vc.ru/finance/343376-nft-dlya-novichkov',
  'summary': 'Что вы можете купить как NFT? NFT могут быть чем угодно, но я покупаю NFT, которые используются в качестве аватаров. Например, они купили NFT за 100 долларов, а через 2 дня продали за 600 долларов. Где купить NFT? 10 лучших торговых площадок NFT 1. Opensea.io OpenSea — лидер продаж NFT. Важность использования кошелька NFT Кошелек NFT не работает как физический кошелек.

In [57]:
# habr
def get_habr_insights(trend: str) -> list[dict]:
    """
    Get insights from habr
    :param trend: trend
    :return: list of insights
    """
    insights = []
    html = get_html(f"https://habr.com/ru/search/?target_type=posts&q={trend}&order=relevance")
    if html:
        soup = BeautifulSoup(html, "html.parser")
        insights = []
        insights_html = soup.find("div", {'class':'tm-articles-list'}).findChildren("article", {'class':'tm-articles-list__item'}, recursive=False)
        logging.info(f"habr: {len(insights)} insights found.")
        print(len(insights_html))
        i = len(insights_html)
        for item in insights_html:
            title_elem = item.find("a", class_="tm-article-snippet__title-link")
            insights.append(
                {
                    "source": "habr.com",
                    "url": "https://habr.com/" + title_elem.get("href"),
                    "title": title_elem.text,
                    "summary": '', # habr is on lazyload
                }
            )
    return insights

get_habr_insights("nft")

20


[{'source': 'habr.com',
  'url': 'https://habr.com//ru/company/ruvds/blog/653085/',
  'title': 'Отдаю NFT за 880 миллионов долларов',
  'summary': ''},
 {'source': 'habr.com',
  'url': 'https://habr.com//ru/company/ruvds/blog/654537/',
  'title': 'Про NFT и деньги',
  'summary': ''},
 {'source': 'habr.com',
  'url': 'https://habr.com//ru/company/ruvds/blog/653665/',
  'title': 'NFT, NFT, NFT — медиавирус',
  'summary': ''},
 {'source': 'habr.com',
  'url': 'https://habr.com//ru/post/684008/',
  'title': 'Из Warcraft в реальную жизнь или что такое Soulbound NFT',
  'summary': ''},
 {'source': 'habr.com',
  'url': 'https://habr.com//ru/post/564676/',
  'title': 'Что такое NFT на самом деле, сколько обмана может скрывать, и в каких сферах реально применим',
  'summary': ''},
 {'source': 'habr.com',
  'url': 'https://habr.com//ru/post/579908/',
  'title': 'Non-fungible token (NFT): основы',
  'summary': ''},
 {'source': 'habr.com',
  'url': 'https://habr.com//ru/company/jugru/blog/651067/'

In [63]:
# Get all insights
def get_insights(trend: str, limit_per_source: int = 0, overall_limit: int = 0) -> list[dict]:
    """
    Get insights from all sources
    :param trend: trend
    :return: list of insights
    """
    insights = []
    sources = [
        get_vc_insights,
        get_habr_insights,
    ]
    if limit_per_source == 0:
        limit_per_source = -1
    if overall_limit == 0:
        overall_limit = -1
    for source in sources:
        insights.extend(source(trend)[:limit_per_source])
    return insights[:overall_limit]

get_insights("nft", limit_per_source=2, overall_limit=3)

20


[{'source': 'vc.ru',
  'title': 'Что такое NFT',
  'url': 'https://vc.ru/crypto/214497-chto-takoe-nft',
  'summary': 'Cryptopunks - первый NFT коллектибл LarvaLabs Интерфейс MetaMaskNFT называют главным трендом 2021 года в блокчейне. Новостей из мира NFT все больше. Сделаю обзор топовых NFT проектов. Накину план, какие сделать первые шаги в NFT. NFT Окей, раз это аббревиатура, давайте начнем с расшифровки.  NFT = Non Fungible Token. Так и появились NFT. Теперь, благодаря NFT — нет.Что такое NFT'},
 {'source': 'vc.ru',
  'title': 'Разбор: что за NFT-игра STEPN, которая даёт зарабатывать на ходьбе, чего опасаться и можно ли на ней заработать',
  'url': 'https://vc.ru/crypto/401206-razbor-chto-za-nft-igra-stepn-kotoraya-daet-zarabatyvat-na-hodbe-chego-opasatsya-i-mozhno-li-na-ney-zarabotat',
  'summary': 'NFT-кроссовки обладают характеристиками, такими как: Класс — Walker, Jogger, Runner и Trainer. Показывает, насколько NFT-предмет прокачен. Так называемый Shoe mint. Масштабы продвижения 

In [66]:
from bs4 import BeautifulSoup
import requests
import logging


# Parser constants
LOGGING_LEVELS = {
    "debug": logging.DEBUG,
    "info": logging.INFO,
    "warning": logging.WARNING,
    "error": logging.ERROR,
    "critical": logging.CRITICAL,
}

logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    level=LOGGING_LEVELS["info"],
)


def get_html(url: str) -> str:
    """
    Get html text from url
    :param url: url
    :return: html text
    """
    try:
        r = requests.get(url)
        r.raise_for_status()
        return r.text
    except (requests.RequestException, ValueError):
        logging.error("Network error")
        return ""


# vc.ru
def get_vc_insights(trend: str) -> list[dict]:
    """
    Get insights from vc.ru
    :param trend: trend
    :return: list of insights
    """
    insights = []
    html = get_html(f"https://vc.ru/search/v2/content/relevant?query={trend}")
    if html:
        soup = BeautifulSoup(html, "html.parser")
        insights = []
        insights_html = soup.find("div", {'class':'search-feed'}).findChildren("div", {'class':'content-feed'}, recursive=False)
        logging.info(f"vc: {len(insights)} insights found.")
        for item in insights_html:
            insights.append(
                {
                    "source": "vc.ru",
                    "title": item.find("div", class_="content-title").text.replace("\n", "").strip(),
                    "url": item.find("a", class_="content-link").get("href"),
                    "summary": item.find("p").text.replace("\xa0", " ").replace("<mark>", " ").replace("</mark>", " ").strip(),
                }
            )
    return insights


get_vc_insights("nft")[:2]

# habr
def get_habr_insights(trend: str) -> list[dict]:
    """
    Get insights from habr
    :param trend: trend
    :return: list of insights
    """
    insights = []
    html = get_html(f"https://habr.com/ru/search/?target_type=posts&q={trend}&order=relevance")
    if html:
        soup = BeautifulSoup(html, "html.parser")
        insights = []
        insights_html = soup.find("div", {'class':'tm-articles-list'}).findChildren("article", {'class':'tm-articles-list__item'}, recursive=False)
        logging.info(f"habr: {len(insights)} insights found.")
        for item in insights_html:
            title_elem = item.find("a", class_="tm-article-snippet__title-link")
            insights.append(
                {
                    "source": "habr.com",
                    "url": "https://habr.com/" + title_elem.get("href"),
                    "title": title_elem.text,
                    "summary": '', # habr is on lazyload
                }
            )
    return insights


# Get all insights
def get_insights(trend: str, limit_per_source: int = 0, overall_limit: int = 0) -> list[dict]:
    """
    Get insights from all sources
    :param trend: trend
    :return: list of insights
    """
    insights = []
    sources = [
        get_vc_insights,
        get_habr_insights,
    ]
    if limit_per_source == 0:
        limit_per_source = -1
    if overall_limit == 0:
        overall_limit = -1
    for source in sources:
        insights.extend(source(trend)[:limit_per_source])
    return insights[:overall_limit]


if __name__ == "__main__":
    print(get_insights("nft", limit_per_source=2, overall_limit=3), sep="\n")


[{'source': 'vc.ru', 'title': 'Что такое NFT', 'url': 'https://vc.ru/crypto/214497-chto-takoe-nft', 'summary': 'Cryptopunks - первый NFT коллектибл LarvaLabs Интерфейс MetaMaskNFT называют главным трендом 2021 года в блокчейне. Новостей из мира NFT все больше. Сделаю обзор топовых NFT проектов. Накину план, какие сделать первые шаги в NFT. NFT Окей, раз это аббревиатура, давайте начнем с расшифровки.  NFT = Non Fungible Token. Так и появились NFT. Теперь, благодаря NFT — нет.Что такое NFT'}, {'source': 'vc.ru', 'title': 'Разбор: что за NFT-игра STEPN, которая даёт зарабатывать на ходьбе, чего опасаться и можно ли на ней заработать', 'url': 'https://vc.ru/crypto/401206-razbor-chto-za-nft-igra-stepn-kotoraya-daet-zarabatyvat-na-hodbe-chego-opasatsya-i-mozhno-li-na-ney-zarabotat', 'summary': 'NFT-кроссовки обладают характеристиками, такими как: Класс — Walker, Jogger, Runner и Trainer. Показывает, насколько NFT-предмет прокачен. Так называемый Shoe mint. Масштабы продвижения потрясают Пер